In [0]:
import opengm
import numpy as np
from operator import itemgetter

import sys
import logging
logging.basicConfig(level=logging.DEBUG,
                    format='[%(levelname)s] (%(threadName)-10s) %(message)s',
                    stream=sys.stdout 
)

class FactorGraph(object):
    
    def __init__(self, variables, operator='multiplier'):
        """
        Factor Graph Class
        :param variables: list of tuples mapping graph variable names
                          to dimensionality.
                          For example, to define 2 binary variables:
                          [
                             ('A', 2),
                             ('B', 2)
                          ]
        :param operator: Factor graph operator, leave as 'multiplier'
        """
        assert isinstance(variables, list)
        
        self.var_names, dimensionality = zip(*variables)
        self.var_names = list(self.var_names)
        dimensionality = list(dimensionality)

        # create factor graph with opengm
        self.gm = opengm.graphicalModel(dimensionality, 
                                        operator=operator)
        
        self.inference = None
        
    
    def add_factor_function(self, variables, probabilities):
        """
        Add a factor function to the factor graph
        
        :param variables: variables to connect in the factor graph
         NOTE: Variables must be specified in the same order as
               the initialization function of the Factor Graph
               
        :probabilities: Probability table for factor function
        """
        
        # convert probability list to np.array
        if not isinstance(probabilities, np.ndarray):
            probabilities = np.array(probabilities)
        
        # if a single variable is specified,
        # convert it to a list
        if not isinstance(variables, list):
            variables = [variables]
        
        # convert variable names to indices
        variables = [self.var_names.index(v) for v in variables]
        
        # add factor function to graph
        self.gm.addFactor(self.gm.addFunction(probabilities),
                          variables)
        
        # reset the inference
        self.inference = None
    
    def infer(self):
        """
        Run inference on the defined factor graph
        """
        self.inference = opengm.inference.BeliefPropagation(self.gm, accumulator='maximizer')
        self.inference.infer()
        
    def get_argmax(self):
        """
        Return index of state with maximum probability
        """
        if not self.inference:
            self.infer()
            
        argmax = self.inference.arg()
        
        return dict((vn, argmax[i]) for i, vn in enumerate(self.var_names))
    
    def get_marginals(self, marginal_vars):
        """
        Get marginal probabilities for specified variables
        """
        if not isinstance(marginal_vars, list):
            marginal_vars = [marginal_vars]
        
        if not self.inference:
            self.infer()
            
        
        marginal_probabilities =  self.inference.marginals(range(len(self.var_names)))
        marginals_ret = {}
        for v in marginal_vars:
            i = self.var_names.index(v)
            marginals_v = marginal_probabilities[i]
            marginals_v /= np.sum(marginals_v)
            marginals_ret[v] = marginals_v
            
        return marginals_ret
    



In [0]:
ATTACK_STATES_MAP = {
    'benign': 1,
    'discovery': 2,
    'access': 3,
    'lateral_movement': 4,
    'privilege_escalation': 5,
    'persistence': 6,
    'defense_evasion': 7,
    'collection': 8,
    'exfiltration': 9,
    'command_control': 10,
    'execution': 11
}


ACTIONS = {
    # each value in an actions' vector corresponds to an attack stage
    'NO-OP':   [1.,   0.61, 0.69, 0.09, 0.2 , 0. ,  0.,   0.,   0. ,  0. ,  0.  ],
    'MONITOR': [0.  , 0.39, 0.31 ,0.84, 0.63, 0.7,  0.07 ,0.1 , 0. ,  0. ,  0.  ],
    'STOP':    [0.  , 0.,   0.  , 0.07, 0.17, 0.3,  0.93 ,0.9 , 1. ,  1. ,  1.  ]
}


def get_prob(stages, p, q):
    assert len(p) == len(q) == len(stages)
    prob = np.zeros(11)
    for i in range(len(p)):
        stage_idx = ATTACK_STATES_MAP[stages[i]] - 1
        prob[stage_idx] = q[i] * (1 - p[i])
    # convert to an 1 x 11 matrix
    return np.array(prob)
    


In [0]:
"""
As an example, we provide the Factor Graph at t=1 with several blanks for you to fill in
Your task is to come up with a general Factor Graph model 
that is parametrized for some general time t
"""
# our sequence of events is simply ['scan']


###################################################
# TODO: Fill in the dimension value for stage S1  #
###################################################
m = FactorGraph([('S1', 11)])
############
# END TODO #
############


# add the f_1 factor function 
###############################################################################
# TODO: Fill in the p, q values for discovery and benign stages respectively  #
###############################################################################
m.add_factor_function('S1', get_prob(['discovery', 'benign'], [0.03, 0.52], [0.5, 0.5]))
############
# END TODO #
############


# run inference
m.infer()

# get argmax and S1 marginal probabilities
######################################################################################################
# TODO: Call the function in class FactorGraph that returns index of state with maximum probability  #
######################################################################################################
argmax = m.get_argmax()
############
# END TODO #
############

marginal_S1 = m.get_marginals('S1')

print(argmax)
print(marginal_S1)

# argmax(marginal_S1) = 1, which represents the discovery attack state (2nd position in array)

# to determine the action to be taken, we look at the probability values
# for the discovery stage for all posible actions, and pick the action
# with the maximum probability
idx = argmax['S1']
action_probabilities = [(k, stage_list[idx]) for k, stage_list in ACTIONS.items()]
print(action_probabilities)

#####################################################################
# TODO: Provide the argmax action corresponding to the argmax stage #
#####################################################################

    
max_action = action_probabilities[np.argmax([x[1] for x in action_probabilities])][0]
############
# END TODO #
############

#max_action, max_probability = max(action_probabilities, key=itemgetter(1))
print(max_action)

{'S1': 1}
{'S1': array([ 0.33103448,  0.66896552,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ])}
[('STOP', 0.0), ('MONITOR', 0.39), ('NO-OP', 0.61)]
NO-OP


# TASK 3.1 

**FACTOR GRAPH FOR T=1 to T=9**

In [0]:
# HINT: Since you only require the argmax of the ACTION dictionary at each stage,
#       convert the dictionary to a list of actions indexed by stage instead
Dictionary={'scan':[['discovery','benign'],[0.03,0.52],[0.5,0.5]], 
            'login':[['benign'], [0.01],[0.5]],
            'sensitive_uri':[['privilege_escalation', 'benign'],[0.015, 0.015],[0.09, 0.11]],
            'new_kernel_module':[['persistence', 'benign'],[0.005, 0.03],[0.07, 0.09]],
           'dns_tunneling': [['benign', 'exfiltration'], [0.08, 0.004],[0.16, 0.18]]
           }

full_sequence = [['S1','scan'],
       ['S2', 'login'],
       ['S3','sensitive_uri'],
       ['S4','sensitive_uri'],
       ['S5','sensitive_uri'],
       ['S6','new_kernel_module'],
       ['S7','dns_tunneling'],
       ['S8','dns_tunneling'],
       ['S9','dns_tunneling']]

In [0]:
m = FactorGraph([('S1', 11),('S2', 11),('S3', 11),('S4', 11),('S5', 11),('S6', 11),('S7', 11),('S8', 11),('S9', 11)])

In [0]:
"""
Build your general model below.
Run your inference for t=1 through t=9
"""
#full_sequence = ['scan', 'login', 'sensitive_uri', 'sensitive_uri', 'sensitive_uri',
#                 'new_kernel_module', 'dns_tunneling', 'dns_tunneling', 'dns_tunneling']

for seq in full_sequence:
    if seq[0] == 'S6':
        m.add_factor_function(seq[0], get_prob(Dictionary[seq[1]][0], Dictionary[seq[1]][1], Dictionary[seq[1]][2])+get_prob(['persistence'],[0.006],[0.15]))
    elif seq[0] == 'S5':
        m.add_factor_function(seq[0], get_prob(Dictionary[seq[1]][0], Dictionary[seq[1]][1], Dictionary[seq[1]][2])+get_prob(['privilege_escalation'],[0.05],[0.15]))
    else:
        m.add_factor_function(seq[0], get_prob(Dictionary[seq[1]][0], Dictionary[seq[1]][1], Dictionary[seq[1]][2]))


m.infer()
argmax = m.get_argmax()

marginals = []
action_probabilities = []

for seq in full_sequence:
    marginals.append(m.get_marginals(seq[0]))
    
for seq in full_sequence:
    idx = argmax[seq[0]]
    action_probabilities.append([(k, stage_list[idx]) for k, stage_list in ACTIONS.items()])



#####################################################################
# TODO: Provide the argmax action corresponding to the argmax stage #
#####################################################################
max_action = []

for elem in action_probabilities:
    max_action.append(elem[np.argmax([x[1] for x in elem])][0])

    


#max_action = action_probabilities[np.argmax([x[1] for x in action_probabilities])][0]


# TASK 3.2

**Marginal probabilities are:**

In [0]:
for p in marginals:
    print(p)

{'S1': array([ 0.33103448,  0.66896552,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ])}
{'S2': array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])}
{'S3': array([ 0.55,  0.  ,  0.  ,  0.  ,  0.45,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ])}
{'S4': array([ 0.55,  0.  ,  0.  ,  0.  ,  0.45,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ])}
{'S5': array([ 0.3191458,  0.       ,  0.       ,  0.       ,  0.6808542,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,  0.       ])}
{'S6': array([ 0.28524751,  0.        ,  0.        ,  0.        ,  0.        ,
        0.71475249,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ])}
{'S7': array([ 0.45086988,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.54913012,  0.        ,  0.        ])}
{'S8': array([ 0.45086988,  0.        ,  0.        ,  0.        ,  0.       

In [0]:
inv_map = {v: k for k, v in ATTACK_STATES_MAP.iteritems()}

**The corresponding most probable states are:**

In [0]:
for p in marginals:
    print(inv_map[np.argmax(p.values())+1])

discovery
benign
benign
benign
privilege_escalation
persistence
exfiltration
exfiltration
exfiltration


# TASK 3.3

**We take the argmax to recommend an action for each step. We arrived at this decision as we had an array of probabilities, and wanted to choose the action which would have the maximum probability**

In [0]:
print('actions after argmax are:', max_action)

('actions after argmax are:', ['NO-OP', 'NO-OP', 'NO-OP', 'NO-OP', 'MONITOR', 'MONITOR', 'STOP', 'STOP', 'STOP'])


# TASK 3.4

**The earliest stage  at which the model recommends the 'STOP' action is at S7**

# TASK 3.5

**NEW FACTOR GRAPH AS FOLLOWS**

In [0]:
full_sequence2 = [['S1','scan'],
       ['S2', 'login'],
       ['S3','sensitive_uri'],
       ['S4','sensitive_uri'],
       ['S5','sensitive_uri'],
       ['S6','new_kernel_module'],
       ['S8','dns_tunneling'],
       ['S9','dns_tunneling']]

g = FactorGraph([('S1', 11),('S2', 11),('S3', 11),('S4', 11),('S5', 11),('S6', 11),('S8', 11),('S9', 11)])

In [0]:
"""
Build your general model below.
Run your inference for t=1 through t=9
"""
#full_sequence = ['scan', 'login', 'sensitive_uri', 'sensitive_uri', 'sensitive_uri',
#                 'new_kernel_module', 'dns_tunneling', 'dns_tunneling', 'dns_tunneling']

for seq in full_sequence2:
    if seq[0] == 'S6':
        #print('got')
        g.add_factor_function(seq[0], get_prob(Dictionary[seq[1]][0], Dictionary[seq[1]][1], Dictionary[seq[1]][2])+get_prob(['persistence'],[0.006],[0.15]))
    elif seq[0] == 'S5':
        g.add_factor_function(seq[0], get_prob(Dictionary[seq[1]][0], Dictionary[seq[1]][1], Dictionary[seq[1]][2])+get_prob(['privilege_escalation'],[0.05],[0.15]))
    else:
        g.add_factor_function(seq[0], get_prob(Dictionary[seq[1]][0], Dictionary[seq[1]][1], Dictionary[seq[1]][2]))


g.infer()
argmax = g.get_argmax()

marginals = []
action_probabilities = []

for seq in full_sequence2:
    marginals.append(g.get_marginals(seq[0]))
    
for seq in full_sequence2:
    idx = argmax[seq[0]]
    action_probabilities.append([(k, stage_list[idx]) for k, stage_list in ACTIONS.items()])



#####################################################################
# TODO: Provide the argmax action corresponding to the argmax stage #
#####################################################################
max_action = []

for elem in action_probabilities:
    max_action.append(elem[np.argmax([x[1] for x in elem])][0])

    
print(max_action)
#max_action = action_probabilities[np.argmax([x[1] for x in action_probabilities])][0]


['NO-OP', 'NO-OP', 'NO-OP', 'NO-OP', 'MONITOR', 'MONITOR', 'STOP', 'STOP']


In [0]:
for d in marginals:
    print(d)

{'S1': array([ 0.33103448,  0.66896552,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ])}
{'S2': array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])}
{'S3': array([ 0.55,  0.  ,  0.  ,  0.  ,  0.45,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ])}
{'S4': array([ 0.55,  0.  ,  0.  ,  0.  ,  0.45,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ])}
{'S5': array([ 0.3191458,  0.       ,  0.       ,  0.       ,  0.6808542,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,  0.       ])}
{'S6': array([ 0.28524751,  0.        ,  0.        ,  0.        ,  0.        ,
        0.71475249,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ])}
{'S8': array([ 0.45086988,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.54913012,  0.        ,  0.        ])}
{'S9': array([ 0.45086988,  0.        ,  0.        ,  0.        ,  0.       

**The marginals stay the same as removing e7 and s7 does not affect any other state as there is no factor function connecting it to any other one**

# TASK 3.6

**HMM model in pdf**

**The parameters needed for the HMM model are:Transition state matrix, Emission state matrix, Initial probabilities**

**One advantage of Factor graphs over the HMM is that it is more general, i.e., HMM has the markov assumption considering that the future state only depends on the present state. There is no such assumption in Factor graphs and we can have factor functions for modelling any number of dependencies**

# TASK 4.0

**It is not possible to this attack using only one event as we need the whole sequence of events**

**Scan: If we get the probability of Discovery higher than Benign but it's a legitimate user
<br>
Login: Cannot get a false positive as there is no other state
<br>
Sensitive URI:  If we get the probability of privilege escalation higher than Benign but it's a legitimate user
<br>
New Executable File: If we get the probability of persistence higher than Benign but it's a legitimate user
<br>
Homepage overwritten with a new link: If we predict command and control or execution but it's Benign and a legitimate user
<br>
Webserver restarted: If we predict command and control or execution but it's Benign and a legitimate user**

# TASK 4.1

**Visual representation in pdf**

# TASK 4.2

**The variables Scan, Login and Sensitive URI and the corresponding factor functions (f1, f2, f3, f4, f5 and r) are the same as the previous factor graph**